In [1]:
import pandas as pd
from tbdynamics.plotting import plot_model_vs_actual
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.calibration.utils import get_bcm, load_targets
from multiprocessing import cpu_count

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
pd.options.plotting.backend = "plotly"

In [3]:
params = {
    "start_population_size": 2000000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
covid_effects = {
    'detection_reduction':True,
    'contact_reduction':True
}
bcm = get_bcm(params, covid_effects)

In [4]:
# Set the number of workers for parallel optimization
orunner = optimize_model(bcm, opt_class=ng.optimizers.TwoPointsDE, num_workers=cpu_count())
for i in range(8):
    rec = orunner.minimize(1000)
mle_params = rec.value[1]
mle_params

{'contact_rate': 0.024146713916783653,
 'rr_infection_latent': 0.14811150798330036,
 'rr_infection_recovered': 0.19533953494412504,
 'progression_multiplier': 0.9065530194518613,
 'smear_positive_death_rate': 0.3817222741511205,
 'smear_negative_death_rate': 0.019727340605187136,
 'smear_positive_self_recovery': 0.23010662562604406,
 'smear_negative_self_recovery': 0.14601379351904492,
 'screening_scaleup_shape': 0.4728432464250723,
 'screening_inflection_time': 2001.0459139690909,
 'time_to_screening_end_asymp': 1.577920710255754,
 'contact_reduction': 0.13571822985026724,
 'detection_reduction': 0.2686197936714753,
 'notif_dispersion': 8410.713909749278,
 'prev_dispersion': 39.37840703586491}

In [5]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets()

In [6]:
plot_model_vs_actual(
    derived_df_0, targets['total_population'], "total_population", "Population", "Modelled vs Data"
)

In [7]:
plot_model_vs_actual(derived_df_0, targets['adults_prevalence_pulmonary_target'], 'adults_prevalence_pulmonary', 'Infectious prevalence', 'Modelled vs Estimation from 2017 prevalence survey')

In [8]:
plot_model_vs_actual(
    derived_df_0, targets['incidence_target'], "incidence", "Incidence", "Modelled vs Data"
)

In [9]:
plot_model_vs_actual(
    derived_df_0, targets['notification'], "notification", "Notifications", "Modelled vs Data"
)

In [11]:
derived_df_0['detection_rate'].plot()